## Approach

   1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. Collection processed in this notebook is ODIAC CO₂ emissions version 2022.
   2. Pass the STAC item into raster API `/stac/tilejson.json` endpoint
   3. We'll visualize two tiles (side-by-side) allowing for comparison of each of the time points using `folium.plugins.DualMap`
   4. After the visualization, we'll perform zonal statistics for a given polygon.
   

## About the Data

The Open-Data Inventory for Anthropogenic Carbon dioxide (ODIAC) is a high-spatial resolution global emission data product of CO₂ emissions from fossil fuel combustion (Oda and Maksyutov, 2011). ODIAC pioneered the combined use of space-based nighttime light data and individual power plant emission/location profiles to estimate the global spatial extent of fossil fuel CO₂ emissions. With the innovative emission modeling approach, ODIAC achieved the fine picture of global fossil fuel CO₂ emissions at a 1x1km.

# Installing the required libraries.
Please run the next cell to install all the required libraries to run the notebook.

## Querying the STAC API

In [83]:
import requests
import folium
import folium.plugins
from folium import Map, TileLayer 
from pystac_client import Client 
import pandas as pd
import matplotlib.pyplot as plt
import branca.colormap as cm
import geopandas
from pyproj import Geod
from shapely import wkt
import seaborn as sns
import glob
import numpy as np
from datetime import datetime

In [85]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://ghg.center/api/stac"
RASTER_API_URL = "https://ghg.center/api/raster"

STAC_API_URL_veda = "https://staging-stac.delta-backend.com"
RASTER_API_URL_veda = "https://staging-raster.delta-backend.com"

#Please use the collection name similar to the one used in STAC collection.
# Name of the collection for ODIAC dataset. 
collection_name__odiac = "odiac-ffco2-monthgrid-v2022"
collection_name_oco = "oco2geos-co2-daygrid-v10r"
collection_name_no2 = "no2-monthly"


In [86]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection_odiac = requests.get(f"{STAC_API_URL}/collections/{collection_name__odiac}").json()
collection_odiac

collection_oco2 = requests.get(f"{STAC_API_URL}/collections/{collection_name_oco}").json()
collection_oco2

collection_no2 = requests.get(f"{STAC_API_URL_veda}/collections/{collection_name_no2}").json()
collection_no2


{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'}],
 'title': 'NO₂',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01 00:00:00+00',
     '2023-09-30 00:00:00+00']]}},
 'license': 'MIT',
 'keywords': None,
 'providers': None,
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2023-09-30T00:00:00Z']},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of NO₂ and less acti

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2000 to December 2021. By looking at the `dashboard:time density` we observe that the periodic frequency of these observations is monthly. 

In [87]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [88]:
# Check total number of items available
number_of_items = get_item_count(collection_name__odiac)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name__odiac}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

number_of_items1 = get_item_count(collection_name_oco)
items1 = requests.get(f"{STAC_API_URL}/collections/{collection_name_oco}/items?limit={number_of_items1}").json()["features"]
print(f"Found {len(items1)} items")

response = requests.post(
    f"{STAC_API_URL_veda}/search",
    json={
        "collections": [collection_name_no2],
        "query": {"datetime": {"eq": "2021-01-01T00:00:00"}},
        "limit": 100,
    },
).json()
itemsv = response["features"]
print(len(itemsv))

Found 264 items
Found 2615 items
1


This makes sense as there are 22 years between 2000 - 2021, with 12 months per year, meaning 264 records in total.  

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

## Exploring Changes in Carbon Dioxide (CO₂) levels using the Raster API

We will explore changes in fossil fuel emissions in urban egions. In this notebook, we'll explore the impacts of these emissions and explore these changes over time. We'll then visualize the outputs on a map using `folium`. 

In [89]:
# to access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:7]: item for item in items} 
asset_name = "co2-emissions"

items1 = {item["properties"]["datetime"]: item for item in items1} 
asset_name1 = "xco2"

itemv = itemsv[0]

In [90]:
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}
#rescale_values1 = {"max":items1[list(items1.keys())[0]]["assets"][asset_name1]["raster:bands"][0]["histogram"]["max"], "min":items1[list(items1.keys())[0]]["assets"][asset_name1]["raster:bands"][0]["histogram"]["min"]}
rescale_values1 = {'max':415 , 'min': 412}
rescale_valuesv = {'max': 9050373673124971, 'min': 0}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice, once for January 2020 and again for January 2000, so that we can visualize each event independently. 

In [93]:
color_map = "rainbow" # please select the color ramp from matplotlib library.
january_2020_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2020-01']['collection']}&item={items['2020-01']['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
january_2020_tile

color_map1 = "magma"
oco2_1 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items1[list(items1.keys())[400]]['collection']}&item={items1[list(items1.keys())[400]]['id']}"
    f"&assets={asset_name1}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map1}"
    f"&rescale={rescale_values1['min']},{rescale_values1['max']}", 
).json()
oco2_1


no2 = requests.get(
    f"{RASTER_API_URL_veda}/stac/tilejson.json?collection={itemv['collection']}&item={itemv['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=magma"
    f"&rescale={rescale_valuesv['min']},{rescale_valuesv['max']}",
).json()
no2


{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_0.10x0.10_202101_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=magma&rescale=0%2C9050373673124971'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

## Visualizing CO₂ emissions


In [92]:
sa_aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              25.519052777398997,
              -24.8470086420499
            ],
            [
              25.519052777398997,
              -28.145634397543844
            ],
            [
              30.29637465013832,
              -28.145634397543844
            ],
            [
              30.29637465013832,
              -24.8470086420499
            ],
            [
              25.519052777398997,
              -24.8470086420499
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [94]:
from folium.plugins import MousePosition

map_ = folium.Map(location=(-25.943840, 29.789560), zoom_start=7)

map_layer_2020 = TileLayer(
    tiles=january_2020_tile["tiles"][0],
    attr="GHG",
    name ="ODIAC",
    opacity=0.5,
)
map_layer_2020_2 = TileLayer(
    tiles=no2["tiles"][0],
    attr="GHG",
    name = "NO2",
    opacity=0.5,
)
sbs = folium.plugins.SideBySideLayers(layer_left=map_layer_2020, layer_right=map_layer_2020_2)
map_layer_2020.add_to(map_)
map_layer_2020_2.add_to(map_)
folium.GeoJson(sa_aoi, name="louisiana, USA").add_to(map_)
sbs.add_to(map_)
MousePosition().add_to(map_)
# visualising the map
map_

In [95]:
import folium
import folium.plugins

# Set initial zoom and center of map for CO₂ Layer
map_ = folium.Map(location=(-25.943840, 29.789560), zoom_start=7)

# December 2001
map_layer_2020 = TileLayer(
    tiles=january_2020_tile["tiles"][0],
    attr="GHG",
    name ="ODIAC",
    opacity=0.5,
)
map_layer_2020.add_to(map_)

map_layer_2020_1 = TileLayer(
    tiles=oco2_1["tiles"][0],
    attr="GHG",
    name = "OCO2",
    opacity=0.5,
)
map_layer_2020_1.add_to(map_)

map_layer_2020_2 = TileLayer(
    tiles=no2["tiles"][0],
    attr="GHG",
    name = "NO2",
    opacity=0.5,
)
map_layer_2020_2.add_to(map_)

folium.GeoJson(sa_aoi, name="South Africa").add_to(map_)
folium.LayerControl(collapsed=False,position='bottomleft').add_to(map_)

# visualising the map
map_



# Calculating the zonal statistics

In [ ]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name__odiac}/items?limit=500"
).json()["features"]
print(f"Found {len(items)} items")

In [ ]:
# Explore one item to see what it contains
items[0]

In [ ]:
# the bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    return {
        **result["properties"],
        "start_datetime": item["properties"]["start_datetime"][:7],
    }

With the function above we can generate the statistics for the AOI.

In [ ]:
%%time
stats = [generate_stats(item,sa_aoi) for item in items]

In [ ]:
stats[0]

In [ ]:
import pandas as pd

def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["start_datetime"])
    return df

df = clean_stats(stats)
df.head(5)

## Visualizing the Data as a Time Series
We can now explore the ODIAC fossil fuel emission time series available (January 2000 -December 2021) for the Texas, Dallas area of USA. We can plot the data set using the code below:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 10))


plt.plot(
    df["date"],
    df["max"],
    color="red",
    linestyle="-",
    linewidth=0.5,
    label="Max monthly CO₂ emissions",
)

plt.legend()
plt.xlabel("Years")
plt.ylabel("CO2 emissions gC/m2/d")
plt.title("CO2 emission Values for Texas, Dallas (2000-2021)")

In [ ]:
print(items[2]["properties"]["start_datetime"])

In [ ]:
october_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[2]['collection']}&item={items[2]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
october_tile

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=8,
)

map_layer = TileLayer(
    tiles=october_tile["tiles"][0],
    attr="GHG", opacity = 0.5
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary

In this notebook we have successfully explored, analysed and visualized STAC collecetion for ODIAC C02 fossisl fuel emission (2022).